**Create CNN Model and Optimize it using Keras Tuner**

# **Installation of Keras Tuner:**

In [ ]:
!pip install keras-tuner --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


# **Importing Libraries:**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# **Version Check:**

In [ ]:
print("Keras Version (from TensorFlow):", tf.keras.__version__)
print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)  # If using standalone Keras
print("NumPy Version:", np.__version__)

Keras Version (from TensorFlow): 3.8.0
TensorFlow Version: 2.18.0
Keras Version: 3.8.0
NumPy Version: 1.26.4


# **Dataset Loading (Fashion MNIST):**

In [ ]:
# This line of code tells the program to get the Fashion MNIST dataset from Keras and store it in a container labeled "fashion_mnist" for later use.

fashion_mnist=keras.datasets.fashion_mnist

# **Data Preprocessing:**

In [ ]:
# This line essentially fetches the fashion dataset & automatically organizes it into separate sets for training your model and testing how well it works.

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# **Normalization or Scaling of the Image Data:**

This fashion MNIST dataset is grey scale. So, we will scale down image between 0 - 1, where alll the pixel is converted to 0 - 1, so, it help us to quickly train our model

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [ ]:
train_images[0].shape

(28, 28)

Now, when we have 28X28 pixel, we also need to convert this or resize this. It is very very important for CNN model because in the input layer we need to give the how many numbers of images we have, what is the pixel along with the 4 dimmensional array

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
train_images.shape

(60000, 28, 28, 1)

In [ ]:
test_images.shape

(10000, 28, 28, 1)

# **Building the CNN Model Structure:**

In [ ]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(
              hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

# **Importing Keras Tuner Components:**

In [ ]:
# RandomSearch will help us to find the which parameter will be best for this problem statement

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-15-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


# **Initializing the Keras Tuner with RandomSearch:**

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 22m 10s]
val_accuracy: 0.875333309173584

Best val_accuracy So Far: 0.9156666398048401
Total elapsed time: 02h 04m 06s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 112)         │           1,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 64)          │         179,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 30976)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       1,982,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,163,562 (8.25 MB)

 Trainable params: 2,163,562 (8.25 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 914s 540ms/step - accuracy: 0.9498 - loss: 0.1363 - val_accuracy: 0.9078 - val_loss: 0.2753
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 871s 516ms/step - accuracy: 0.9655 - loss: 0.0941 - val_accuracy: 0.9143 - val_loss: 0.2962
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 915s 542ms/step - accuracy: 0.9763 - loss: 0.0647 - val_accuracy: 0.9102 - val_loss: 0.3785
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 905s 532ms/step - accuracy: 0.9838 - loss: 0.0445 - val_accuracy: 0.9097 - val_loss: 0.3861
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 887s 526ms/step - accuracy: 0.9884 - loss: 0.0332 - val_accuracy: 0.9052 - val_loss: 0.4707
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 914s 521ms/step - accuracy: 0.9904 - loss: 0.0276 - val_accuracy: 0.9082 - val_loss: 0.4961
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 920s 520ms/step - accuracy: 0.9911 - loss: 0.0253 - val_accuracy: 0.9130 - val_loss: 0.4800
